In [1]:
# Setup: Add src to path so we can import y0
import sys
import os

# Assuming this notebook is in the 'notebooks' directory, we go up one level to the root
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Added {project_root} to sys.path")

Added /Users/zuck016/Projects/CausalInference/y0-causal-inference/y0/src to sys.path


## 2. Real World Example: `idcd.py`

Let's look at the `idcd` function in `src/y0/algorithm/identify/idcd.py` and how `tests/test_algorithm/test_idcd.py` covers it.

### A. The "Green" Lines (Covered Code)

In `idcd.py`, there is a check for the base case where the ancestral closure equals the target set. This represents a successful identification step.

```python
    # # lines 17-18
    if ancestral_closure == targets:
        logger.debug(f"[{_number_recursions}]: Lines 17-18 - SUCCESS (Ancestral closure = targets)")
        return distribution_a
```

These lines are **covered** (green) because `test_idcd.py` has a test case `test_base_case_ancestral_closure_equals_district` that triggers this condition.

Let's replicate that test case here to see it in action:

In [3]:
from y0.algorithm.identify.idcd import idcd
from y0.dsl import P, X, Y, Z
from y0.graph import NxMixedGraph

# Setup the graph and variables
graph = NxMixedGraph.from_edges(directed=[(X, Y), (Y, Z)])
targets = {Z}
district = {Z}
distribution = P(Z)

# Run the function
# This execution path hits the "if ancestral_closure == targets:" block
result = idcd(
    graph=graph,
    targets=targets,
    district=district,
    distribution=distribution,
)

print(f"Result: {result}")
print("Success! The code path for the base case was executed.")

Result: P(Z)
Success! The code path for the base case was executed.


### B. The "Red" Lines (Missing Coverage)

Now let's look at a part of `idcd.py` that might show up as **Missing** (red) in a coverage report.

Around line 83 in `idcd.py`, there is a defensive check:

```python
    # TODO - add test for this error case here
    # checking recursive case (must have targets ⊊ ancestral_closure ⊊ district)
    if not (targets < ancestral_closure and ancestral_closure < district):
        raise ValueError(
            f"Unexpected state: expected targets ⊊ ancestral_closure ⊊ district, but got..."
        )
```

This block raises a `ValueError` if the algorithm gets into an unexpected state. Because there is currently no test case in `test_idcd.py` that forces this specific invalid state, these lines are not executed during testing.

In a coverage report, this would appear as a range of missing lines (e.g., `83-87`).

To "fix" this missing coverage, we would need to write a test that constructs a graph where `targets`, `ancestral_closure`, and `district` violate the strict subset relationship expected at this point in the algorithm.

## 3. Real World Example: `test_ioscm.py`

Now let's look at `test_ioscm.py`. This file tests utility functions for Input-Output Structural Causal Models.

One interesting test is `test_simplify_strongly_connected_components_3`. It tests a specific edge case: an undirected edge existing *within* a strongly connected component (SCC).

```python
    def test_simplify_strongly_connected_components_3(self) -> None:
        """Test a utility function to simplify strongly-connected components for a graph.

        This test covers the case where an undirected edge exists between two nodes in
        the same strongly connected component. The edge should be removed during
        simplification since both nodes get collapsed into a single representative node.
        """
```

This test ensures that the code responsible for collapsing SCCs correctly handles (and removes) internal undirected edges. If this test didn't exist, the lines of code handling that specific cleanup might be marked as "Missing" coverage (or worse, the bug wouldn't be caught).

Let's run that simplification logic here:

In [4]:
from y0.algorithm.ioscm.utils import simplify_strongly_connected_components
from y0.dsl import W, X, Y, Z
from y0.graph import NxMixedGraph

# Create a graph with an undirected edge (X, W) inside an SCC {X, W, Z}
graph = NxMixedGraph.from_edges(
    directed=[
        (X, W),
        (W, Z),
        (Z, X),
        (W, Y),
    ],
    undirected=[
        (X, W)  # this undirected edge is within the SCC
    ],
)

print("Original Graph Nodes:", graph.nodes())
print("Original Undirected Edges:", graph.undirected.edges())

# Run the simplification
simplified_graph, result_dict = simplify_strongly_connected_components(graph)

print("\nSimplified Graph Nodes:", simplified_graph.nodes())
print("Simplified Undirected Edges:", simplified_graph.undirected.edges())

# Verify the internal edge is gone
if len(simplified_graph.undirected.edges()) == 0:
    print("\nSuccess! The internal undirected edge was removed as expected.")
else:
    print("\nFailure! The edge remains.")

Original Graph Nodes: [X, W, Z, Y]
Original Undirected Edges: [(X, W)]

Simplified Graph Nodes: [W, Y]
Simplified Undirected Edges: []

Success! The internal undirected edge was removed as expected.


## 4. Coverage Configuration Syntax (`# pragma: no cover`)

Sometimes you have code that *cannot* or *should not* be tested, such as:
*   Defensive assertions that should never happen if the code is correct.
*   Code that only runs on specific operating systems.
*   Abstract methods in base classes.

You can tell the coverage tool to ignore these lines using the comment syntax `# pragma: no cover`.

**Example:**

In `src/y0/algorithm/transport.py`, you can see this syntax in action:

```python
    if isinstance(query.expression, Zero):  # pragma: no cover
```

This line is excluded from coverage calculations. This tells the coverage tool: "I know this line isn't covered, and that's intentional. Don't count it against my score."

## 5. Running Coverage Analysis in the Notebook

You can run the coverage tool directly from this notebook to see the report for yourself. We will use `pytest` with the `pytest-cov` plugin.

We will target `y0.algorithm.identify.idcd` and run the tests in `tests/test_algorithm/test_idcd.py`.

In [18]:
# Install pytest-cov to enable coverage reporting
import sys

# We attempt to install using 'uv' first (robust for venvs without pip).
# If that fails (e.g., uv not installed), we fallback to standard 'pip'.
!uv pip install --python "{sys.executable}" pytest-cov || "{sys.executable}" -m pip install pytest-cov

Using Python 3.12.9 environment at: /Users/zuck016/Projects/CausalInference/y0-causal-inference/y0/.venv
Audited 1 package in 49ms


In [19]:
import sys
import subprocess
import os

# Define the command to run pytest with coverage
# We target the specific file we've been discussing: y0.algorithm.identify.idcd
# And run the relevant tests: tests/test_algorithm/test_idcd.py
# We add --cov-report=xml to generate a machine-readable report
command = [
    sys.executable, "-m", "pytest",
    "--cov=y0.algorithm.identify.idcd",
    "--cov-report=xml",
    "tests/test_algorithm/test_idcd.py"
]

print(f"Running command: {' '.join(command)}\n")

# Run the command and capture output
# We set cwd to the project root (parent of notebooks dir) to ensure paths work correctly
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

result = subprocess.run(command, capture_output=True, text=True, cwd=project_root)

# Print the output
# stdout usually contains the test session progress
# stderr usually contains the coverage report
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

Running command: /Users/zuck016/Projects/CausalInference/y0-causal-inference/y0/.venv/bin/python -m pytest --cov=y0.algorithm.identify.idcd --cov-report=xml tests/test_algorithm/test_idcd.py

STDOUT: ============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-9.0.2, pluggy-1.6.0
rootdir: /Users/zuck016/Projects/CausalInference/y0-causal-inference/y0
configfile: pyproject.toml
plugins: anyio-4.12.0, cov-7.0.0
collected 24 items

tests/test_algorithm/test_idcd.py .F...............FF.....               [100%]

=================================== FAILURES ===================================
____ TestValidatePreconditions.test_district_not_consolidated_raises_error _____

self = <tests.test_algorithm.test_idcd.TestValidatePreconditions testMethod=test_district_not_consolidated_raises_error>

    def test_district_not_consolidated_raises_error(self) -> None:
        """District must satisfy CD(G_D) = {D} (single consolidated distr

### Analyzing the Output

Look at the output above. You should see a table similar to the one we discussed in Section 1.

*   **Stmts**: The total statements in `idcd.py`.
*   **Miss**: The number of statements missed.
*   **Cover**: The percentage covered.

If you see `Missing` lines (e.g., `83-87`), those correspond to the defensive check we identified earlier as lacking a test case.

### Automated Interpretation

The raw output above can be dense. Run the cell below to parse the output and get a human-readable summary of the coverage status and test results.

In [20]:
import xml.etree.ElementTree as ET
import os
import re

def analyze_results(stdout_output):
    print("=" * 40)
    print("AUTOMATED ANALYSIS")
    print("=" * 40)

    # 1. Analyze Test Success/Failure (from STDOUT)
    if "FAILED" in stdout_output:
        print(f"❌ Tests Failed!")
        failure_matches = re.findall(r"FAILED (.*?) - (.*)", stdout_output)
        for test, msg in failure_matches:
            short_test = test.split("::")[-1]
            print(f"  • {short_test}: {msg[:100]}...")
    else:
        print("✅ All tests passed.")

    print("-" * 40)

    # 2. Analyze Coverage (from coverage.xml)
    # We look for the coverage.xml file generated in the project root
    # (parent of notebooks dir)
    xml_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), "..")), "coverage.xml")
    
    if not os.path.exists(xml_path):
        print(f"⚠️ coverage.xml not found at {xml_path}")
        print("Did you run the previous cell with --cov-report=xml?")
        return

    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # Find the class element for idcd.py
        target_file = "idcd.py"
        file_node = None
        
        # Search for the file in the XML
        for cls in root.findall(".//class"):
            if target_file in cls.get("filename"):
                file_node = cls
                break
        
        if file_node is not None:
            filename = file_node.get("filename")
            line_rate = float(file_node.get("line-rate"))
            print(f"Coverage Report for: {filename}")
            print(f"Coverage: {line_rate:.1%}")
            
            # Find missing lines (hits="0")
            missing_lines = []
            for line in file_node.findall("lines/line"):
                if int(line.get("hits")) == 0:
                    missing_lines.append(int(line.get("number")))
            
            print(f"Missing Lines: {missing_lines}")
            
            # Check for defensive check (approx lines 83-87)
            # We check if any line in the 80s is missing
            defensive_check_lines = set(range(80, 90))
            missing_set = set(missing_lines)
            
            intersection = missing_set.intersection(defensive_check_lines)
            
            if intersection:
                print(f"\n🔍 INSIGHT: Lines {sorted(list(intersection))} are missing coverage.")
                print("   This confirms that the defensive check (ValueError path) is untested.")
            else:
                print("\n🔍 INSIGHT: The defensive check appears to be covered.")
                
        else:
            print(f"Could not find entry for {target_file} in coverage.xml")
            
    except Exception as e:
        print(f"Error parsing XML: {e}")

# Run the analysis
if 'result' in locals():
    analyze_results(result.stdout)
else:
    print("No result variable found. Did you run the previous cell?")

AUTOMATED ANALYSIS
❌ Tests Failed!
----------------------------------------
Coverage Report for: src/y0/algorithm/identify/idcd.py
Coverage: 92.9%
Missing Lines: [80, 81, 88, 150, 228]

🔍 INSIGHT: Lines [80, 81, 88] are missing coverage.
   This confirms that the defensive check (ValueError path) is untested.
